In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from sklearn import svm, cross_validation
import pylab as pl
from PIL import Image
import numpy as np
import os

In [ ]:
# Make sure that caffe is on the python path:
caffe_root = '/u/zexuan/caffe/caffe/'  # this file is expected to be in {caffe_root}/examples
caffe_real_root = '/pkgs/caffe/'
thesis_root = '/ais/gobi2/pingpong/thesis/'
#!ls /pkgs/caffe
import sys
sys.path.insert(0, caffe_real_root + 'python')
import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

if not os.path.isfile('/u/zexuan/caffe/caffe/models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print("Downloading pre-trained CaffeNet model...")
    !~/caffe/caffe/scripts/download_model_binary.py ~/caffe/caffe/models/bvlc_reference_caffenet

In [ ]:
caffe.set_device(0)
caffe.set_mode_gpu()

In [ ]:
# MODEL_FILE = caffe_root +'models/bvlc_reference_caffenet/deploy.prototxt'
# PRETRAINED = caffe_root +'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
# caffe.set_mode_cpu()
# net = caffe.Classifier(MODEL_FILE, PRETRAINED,
#                        mean=np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1),
#                        channel_swap=(2,1,0),
#                        raw_scale=255,
#                        image_dims=(600, 600))
# print net.blobs['data'].data.shape
# [(k, v.data.shape) for k, v in net.blobs.items()]


# caffe.set_mode_cpu()
print caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
#!ls caffe_root+'models/bvlc_reference_caffenet/deploy.prototxt'
# net = caffe.Net(caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt',
#                 caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
#                 caffe.TEST)

#~/caffe/caffe/models/finetune_flickr_style/deploy.prototxt
net = caffe.Net(caffe_root + 'models/finetune_stroke_cls/deploy.prototxt',
                thesis_root + 'fine_tune_stroke_cls_500iter.caffemodel',
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB
print net.blobs['data'].data.shape
[(k, v.data.shape) for k, v in net.blobs.items()] 

In [ ]:
import skimage
import skimage.io
import time

start_time = time.time()
training_set = {}
training_set['target'] = []
container_path = 'labeled_img_selected_all_classes/'

folders = [f for f in sorted(os.listdir(container_path)) if os.path.isdir(os.path.join(container_path, f))]
print 'folders: ', folders

result = []
for folder in folders:
    folder_path = os.path.join(container_path, folder)
    documents = [os.path.join(folder_path, d) for d in sorted(os.listdir(folder_path))]
    input_10_imgs = []
    for pic in documents:
        if not pic.endswith('png'):
            continue
        img = skimage.img_as_float(skimage.io.imread(pic)).astype(np.float32)
        input_10_imgs.append(img)
        if len(input_10_imgs) >= 10:
            #net.blobs['data'].data[...] = transformer.preprocess('data', training_set['data'][0])
            #net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), training_set['data'][i:i+10])
            net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), input_10_imgs)
            out = net.forward()
            result.append(np.mean(net.blobs['fc6'].data, axis=0))
            input_10_imgs = []
        training_set['target'].append(folder)
#training_set['data']=data
#training_set['target']=target
print 'Total time loading training data: ', time.time()-start_time, ' seconds'
print 'number of images: ', len(training_set['target'])
#print 'dmensionality of each image: ', input_10_imgs[0].shape

In [ ]:
start_time = time.time()
testing_set = {}
testing_set['target'] = []
container_path = 'labeled_img_selected_all_classes_testset/'

folders = [f for f in sorted(os.listdir(container_path)) if os.path.isdir(os.path.join(container_path, f))]
print 'folders: ', folders

for folder in folders:
    folder_path = os.path.join(container_path, folder)
    documents = [os.path.join(folder_path, d) for d in sorted(os.listdir(folder_path))]
    input_10_imgs = []
    for pic in documents:
        if not pic.endswith('png'):
            continue
        img = skimage.img_as_float(skimage.io.imread(pic)).astype(np.float32)
        input_10_imgs.append(img)
        if len(input_10_imgs) >= 10:
            #net.blobs['data'].data[...] = transformer.preprocess('data', training_set['data'][0])
            #net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), training_set['data'][i:i+10])
            net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), input_10_imgs)
            out = net.forward()
            result.append(np.mean(net.blobs['fc6'].data, axis=0))
            input_10_imgs = []
        testing_set['target'].append(folder)
#training_set['data']=data
#training_set['target']=target
print 'Total time loading training data: ', time.time()-start_time, ' seconds'
print 'number of images: ', len(testing_set['target'])

In [ ]:
# print caffe.io.load_image(thesis_root + 'labeled_image_selected/bottom_player_winning_selected/point_00001_frame_00033.png').shape
# print caffe.io.load_image(thesis_root + 'labeled_image_selected/bottom_player_winning_selected/point_00001_frame_00033.png')[0][0]
# print training_set['data'][0][0][0]

'''
result = []
#for i in xrange(2200, 2300,10):
for i in xrange(0, traning_set_size, 10):
    #net.blobs['data'].data[...] = transformer.preprocess('data', training_set['data'][0])
    net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), training_set['data'][i:i+10])
    out = net.forward()
    #print net.blobs['fc7'].data.shape
    result.append(np.mean(net.blobs['fc6'].data, axis=0))
'''

print 'Number of input to SVM: ', len(result), 'Size of each input', result[0].shape

print transformer.deprocess('data', net.blobs['data'].data[0]).shape
print 'Sample image from the training set'
#plt.imshow(transformer.deprocess('data', net.blobs['data'].data[4]))

In [ ]:
import pickle

pickle.dump(result, open("nn_result_fc6_unnormalized_clf_7videos.pickle", "wb" ) )

In [ ]:
#from sklearn.preprocessing import normalize
print len(result)
print np.max(result[0])
print result[0].shape
normalized_result = []
#normalized_result = normalize(result)
for r in result:
    #normalized_result.append(r/np.max(r))
    normalized_result.append((r-np.mean(r))/np.std(r))

In [ ]:
import pickle

pickle.dump(normalized_result, open("nn_result_fc6_normalized_clf_7videos.pickle", "wb" ) )

In [ ]:
print len(normalized_result)
print np.max(normalized_result[0])
print normalized_result[0].shape
print result[0].shape


In [ ]:
from sklearn import decomposition

traning_set_size = len(training_set['target'])
testing_set_size = len(testing_set['target'])
print 'traning_set_size: ', traning_set_size
print 'normalized_result size: ', len(normalized_result)
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(result, training_set['target'][0:traning_set_size:10], test_size=0.25, random_state=42)
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(normalized_result, training_set['target'][0:traning_set_size:10], test_size=0.15, random_state=2)
X_train = normalized_result[:traning_set_size/10]
X_test = normalized_result[traning_set_size/10:]
y_train = training_set['target'][0:traning_set_size:10]
y_test = testing_set['target'][0:traning_set_size:10]


print 'X_train: %d; X_test: %d; y_train: %d; y_test: %d'%(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
from sklearn import svm, cross_validation
X_train = np.asarray(X_train)
svc = svm.SVC(kernel='precomputed')
kernel_train = np.dot(X_train, X_train.T)  # linear kernel
svc.fit(kernel_train, y_train)

# Testing
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
kernel_test = np.dot(X_test, X_train.T)
y_pred = svc.predict(kernel_test)
print accuracy_score(y_test, y_pred)

import pandas as pd
y_true = pd.Series(y_test)
y_pred = pd.Series(y_pred)

#print pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

print confusion_matrix(y_test, y_pred).shape
print confusion_matrix(y_test, y_pred)
#print '\n'.join(sorted(list(set(y_test))))
print len(set(y_test).union(set(y_pred)))

In [ ]:
for c in [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 500]:
    svc = svm.SVC(kernel='linear', C=c) 
    kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
    cv_result = cross_validation.cross_val_score(svc, X_train, y_train, cv=kfold, n_jobs=1)
    print 'C = %g, Cross validation: %s'%(c, cv_result)
    print 'C = %g, Mean value: %g'%(c, np.mean(cv_result))

In [ ]:
for c in [0.01, 0.05, 0.1, 1, 10, 100, 1000, 1000000]:
    svc = svm.SVC(C=c, kernel='poly', degree=1)
    kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
    cv_result = cross_validation.cross_val_score(svc, X_train, y_train, cv=kfold, n_jobs=1)
    print 'C = %g, Cross validation: %s'%(c, cv_result)
    print 'C = %g, Mean value: %g'%(c, np.mean(cv_result))

In [ ]:
for c in [0.01, 0.05, 0.1, 1, 10, 100, 1000, 1000000]:
    svc = svm.SVC(C=c, kernel='poly', degree=2)
    kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
    cv_result = cross_validation.cross_val_score(svc, X_train, y_train, cv=kfold, n_jobs=1)
    print 'C = %g, Cross validation: %s'%(c, cv_result)
    print 'C = %g, Mean value: %g'%(c, np.mean(cv_result))

In [ ]:
for c in [0.01, 0.05, 0.1, 1, 10, 100, 1000, 1000000]:
    svc = svm.SVC(C=c, kernel='poly', degree=3)
    kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
    cv_result = cross_validation.cross_val_score(svc, X_train, y_train, cv=kfold, n_jobs=1)
    print 'C = %g, Cross validation: %s'%(c, cv_result)
    print 'C = %g, Mean value: %g'%(c, np.mean(cv_result))

In [ ]:
clf_best = svm.SVC(kernel='linear', C=0.01).fit(X_train, y_train)
clf_best.score(X_test, y_test)

In [ ]:
print len(X_test)
print len(y_test)
for i,_ in enumerate(X_test):
    if clf_best.predict(X_test[i])[0] == y_test[i]:
        print 'SAME: ', clf_best.predict(X_test[i])[0]
    else:
        print 'DIFFERENT: ', clf_best.predict(X_test[i])[0], y_test[i]
        

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=7)

kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
cv_result = cross_validation.cross_val_score(neigh, X_train, y_train, cv=kfold, n_jobs=1)

print 'Cross validataion result', cv_result, '. Avg:', np.mean(cv_result)
    
clf = neigh.fit(X_train, y_train)
print 'Testing result', clf.score(X_test, y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
cv_result = cross_validation.cross_val_score(gnb, X_train, y_train, cv=kfold, n_jobs=1)
print 'Cross validataion result', cv_result, '. Avg:', np.mean(cv_result)

clf = gnb.fit(X_train, y_train)
print 'Testing result', clf.score(X_test, y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
cv_result = cross_validation.cross_val_score(lr, X_train, y_train, cv=kfold, n_jobs=1)
print 'Cross validataion result', cv_result, '. Avg:', np.mean(cv_result)

clf = lr.fit(X_train, y_train)
print 'Testing result', clf.score(X_test, y_test)

In [ ]:
pickle.dump(clf, open("fc6_normalized_lr_clf_7videos_2.pickle", "wb" ) )

In [ ]:
print sorted(list(set(y_train)))
print len(sorted(list(set(y_train))))